# How Far Deep Learning Systems for Text Detection and Recognition in Natural Scenes are Affected by Occlusion?

### This is a step-by-step guide to generate the ISTD-OC dataset. 

In [ ]:
#Import drive
from google.colab import drive
drive.mount('./gdrive')
%cd "./gdrive/My Drive/Colab Notebooks"


In [ ]:
#check directory
import os
os.chdir(path)

!ls -l

In [ ]:
#generate ISTD-OC crop words for the text recognition task 
import cv2
import random
import numpy as np
from random import randint, gauss, randrange



def readImages(path):
    with open(path,'r',encoding='utf-8-sig') as txt_file:
      for line in txt_file.read().splitlines():
        if "word_"+str(d)+".png" in line:
          coordenadas=line.split(',')
          print(coordenadas[1:])
          return coordenadas[1:]

def gerarCoordenadas(coordenadas):

        result=[]
        regioes=1
        print(regioes)

        for i in range (len(coordenadas)):

          p1 = (int(coordenadas[0]), int(coordenadas[1]))
          p2 = (int(coordenadas[2]), int(coordenadas[3]))
          p3 = (int(coordenadas[4]), int(coordenadas[5]))
          p4 = (int(coordenadas[6]), int(coordenadas[7]))
          result.append(p1)
          result.append(p2)
          result.append(p3)
          result.append(p4)
          
          return result, regioes, coordenadas[-1]
    

def fazLista(coordenadas):
  auxList=[]
  lista=[]
  aux=0
  for i in range (0, len(coordenadas)):
    if aux == 4:
      lista.append(auxList)
      auxList=[]
      aux = 0
    auxList.append(coordenadas[i])
    aux += 1
  lista.append(auxList)
  return lista

def fazDicionario(regioes,lista):
  dic={}
  for i in range(0,regioes):
    dic[str(i)]=lista[i]
    return dic

def regiaoTexto(pathImg, regiaoEscolhida):
  img=cv2.imread(pathImg)
  box=img[regiaoEscolhida[0][1]:regiaoEscolhida[3][1], regiaoEscolhida[0][0]:regiaoEscolhida[1][0]]
  return box

def fazOclusao(box,regiaoEscolhida):
    #RATIO BASED ON IMAGE SIZE TO TEXT BOX SIZE
    alfa=0.9
    rand=random.gauss(0.6,0.05)


    #OCCLUSION SIZE EQUALS BOX SIZE MULTIPLIED BY PROPORTION
    largura_oclusao= int(alfa*box.shape[1]*rand+(1-alfa)*box.shape[1])
    altura_oclusao= int(alfa*box.shape[0]*rand+(1-alfa)*box.shape[0])
    
    #OFFSET
    offset_altura=box.shape[0]-altura_oclusao
    #print(offset_altura)
    offset_largura=box.shape[1]-largura_oclusao
    #print(offset_largura)

    #set occlusion starting point
    max_x_inicial = offset_altura
    max_y_inicial = offset_largura
    ponto_inicial = [random.randint(0,max_x_inicial), random.randint(0,max_y_inicial)]
    
    #add noise to the cut of the image that will be occluded
    gauss = np.random.normal(0,1,img.size)
    gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8') 
    noise_img = cv2.add(img,gauss)
    #cv2_imshow(noise_img)
   
    #set random point original image
    x2_final = noise_img.shape[0]
    y2_final= noise_img.shape[1]
    ponto_aleat_img=[random.randint(0,x2_final), random.randint(0,y2_final)]
    
    #print("proporcao_altura_imagem: ", proporcao_altura_imagem, "proporcao_largura_imagem: ", proporcao_largura_imagem,
     #   "largura_oclusao: ", largura_oclusao, "altura_oclusao: ", altura_oclusao, 
      #  "max_x_inicial: ", max_x_inicial, "max_y_inicial: ", max_y_inicial,
        #"max_x2_final: ", max_x2_final, "max_y2_final: ", max_y2_final,
       # "ponto_inicial: ", ponto_inicial, #"ponto_final: ", ponto_final, 
        #"offset_altura: ", offset_altura,"offset_largura: ",offset_largura, 
        #"regiaoEscolhida:", regiaoEscolhida, "img.shape: ", img.shape,
        #"box.shape: ", box.shape)

    #occlusion region 
    box[ponto_inicial[0]:ponto_inicial[0]+altura_oclusao,ponto_inicial[1]:ponto_inicial[1]+largura_oclusao,:]= noise_img[ponto_aleat_img[0]:ponto_aleat_img[0]+altura_oclusao, ponto_aleat_img[1]:ponto_aleat_img[1]+largura_oclusao]

    return box

for d in range(1,2078):
  pathTxt="base_de_dados/ic15_recognition/coords.txt"
  pathImg="base_de_dados/ic15_recognition/word_"+str(d)+".png"
  coordenadas=readImages(pathTxt)
  c,regioes,transcricao=gerarCoordenadas(coordenadas)
  l=fazLista(c)
  dic=fazDicionario(regioes,l)
  img=cv2.imread(pathImg)

  for i in range(0,regioes):
    regiaoEscolhida=dic[str(i)]
    regiao=regiaoTexto(pathImg,regiaoEscolhida)
      #print("regiao",regiao)
    regiao=fazOclusao(regiao,regiaoEscolhida)
    img[regiaoEscolhida[0][1]:regiaoEscolhida[3][1],regiaoEscolhida[0][0]:regiaoEscolhida[1][0]]=regiao

  pathsave="PSENet/noise_base/recognition/60/img_"+str(d)+".jpg"
  cv2.imwrite(pathsave,img)
